In [1]:
import numpy as np
import copy
import tensorflow as tf
from sklearn import preprocessing
from tensorflow import keras
import os
import pandas as pd
import matplotlib.pyplot as plt
from numpy.random import seed

In [2]:
np.random.seed(1)
df=pd.read_csv(r"rainfall_data.csv")

In [6]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

# load data
data = pd.read_csv('rainfall_data.csv')

# create decision tree for target variable
tree = DecisionTreeClassifier(max_depth=4, random_state=42)

# discretize target variable using decision tree
data['rainfall_tree'] = pd.cut(data['rainfall'], bins=6, labels=False)
tree.fit(data[['rainfall']], data['rainfall_tree'])
data['rainfall_tree'] = tree.predict(data[['rainfall']])

# create new columns for the discretized data
data['windspeed_tree'] = tree.predict(pd.cut(data['windspeed'], bins=6, labels=False).values.reshape(-1,1))
data['tpw_tree'] = tree.predict(pd.cut(data['tpw'], bins=6, labels=False).values.reshape(-1,1))

print(data)


           date  windspeed       tpw   rainfall  rainfall_tree  \
0      6/1/2000  12.245595  0.033285   4.688589              0   
1      6/2/2000  12.825491  0.044874  11.330589              0   
2      6/3/2000  12.921664  0.010243   6.894713              0   
3      6/4/2000  15.149001  0.036881   4.613324              0   
4      6/5/2000  18.495907  0.139491   8.111635              0   
...         ...        ...       ...        ...            ...   
2557  9/26/2020   5.577215  0.009963   4.467977              0   
2558  9/27/2020   5.184293  0.002341   1.572182              0   
2559  9/28/2020   4.469007  0.000867   1.819019              0   
2560  9/29/2020   4.259090  0.001416   2.198017              0   
2561  9/30/2020   5.513838  0.002474  12.713634              0   

      windspeed_tree  tpw_tree  
0                  0         0  
1                  0         0  
2                  0         0  
3                  0         0  
4                  0         0  
...      

C:\Users\shine\anaconda3\envs\environment\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
C:\Users\shine\anaconda3\envs\environment\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [4]:
extreme_data=df[df["rainfall"]>50]
normal_data=df[df["rainfall"]<=50]

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import tensorflow as tf
from keras import backend as K

threshold = 9
best_error = float('inf')
normal_error = 0
extreme_error = 0
import numpy as np
count=0
flag=False


for normal_weight in np.arange(0.005,0.006,0.0001):
    for extreme_weight in np.arange(0.001,0.005, 0.001):
        # Load the data
        data = pd.read_csv('rainfall_data.csv')

        # create decision tree for target variable
        tree = DecisionTreeClassifier(max_depth=4, random_state=42)

        # discretize target variable using decision tree
        data['rainfall_tree'] = pd.cut(data['rainfall'], bins=6, labels=False)
        tree.fit(data[['rainfall']], data['rainfall_tree'])
        data['rainfall_tree'] = tree.predict(data[['rainfall']])

        # create new columns for the discretized data
        data['windspeed_tree'] = tree.predict(pd.cut(data['windspeed'], bins=6, labels=False).values.reshape(-1,1))
        data['tpw_tree'] = tree.predict(pd.cut(data['tpw'], bins=6, labels=False).values.reshape(-1,1))

        count+=1
        print(count)
        # Split the data into normal and extreme rainfall
        normal_data = data[data['rainfall'] <= 50]
        extreme_data = data[data['rainfall'] > 50]
        def weighted_loss(normal_weight, extreme_weight):
            def loss(y_true, y_pred):
                normal_loss = K.mean(K.square(y_true - y_pred))
                extreme_loss = K.mean(K.square(y_true - y_pred) * K.cast(y_true > 50, 'float32'))
                return K.mean(normal_weight * normal_loss + extreme_weight * extreme_loss)
            return loss
        # train your model with the current combination of weights
        # Define a function to find the best regularization strength
        def find_best_reg_strength(train_data, reg_strengths, num_epochs, batch_size):
            best_reg_strength = None
            best_loss = float('inf')
            for reg_strength in reg_strengths:
                # Define the autoencoder architecture
                input_layer = Input(shape=(3,))
                encoded = Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(l2=reg_strength))(input_layer)
                decoded = Dense(3, activation=None)(encoded)

                # Create the autoencoder
                autoencoder = Model(input_layer, decoded)

                # Compile the autoencoder with the weighted loss function
                autoencoder.compile(optimizer='adam', loss=weighted_loss(normal_weight, extreme_weight))

                # Train the model
                history = autoencoder.fit(train_data, train_data, epochs=num_epochs, batch_size=batch_size, validation_split=0.2, verbose=0)

                # Calculate the validation loss
                val_loss = np.mean(history.history['val_loss'])

                # Update the best regularization strength and loss
                if val_loss < best_loss:
                    best_reg_strength = reg_strength
                    best_loss = val_loss

            print('Best regularization strength:', best_reg_strength)
            return best_reg_strength

        # Find the best regularization strength
        reg_strengths = [0.01, 0.1, 1, 10]
        num_epochs = 150
        batch_size = 32
        train_data = normal_data[["windspeed_tree","tpw_tree","rainfall_tree"]].values
        best_reg_strength = find_best_reg_strength(train_data, reg_strengths, num_epochs, batch_size)

        # Define the autoencoder architecture with the best regularization strength
        input_layer = Input(shape=(3,))
        encoded = Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(l2=best_reg_strength))(input_layer)
        decoded = Dense(3, activation=None)(encoded)

        # Create the autoencoder
        autoencoder = Model(input_layer, decoded)

        # Compile the autoencoder with the weighted loss function
        autoencoder.compile(optimizer='adam', loss=weighted_loss(normal_weight, extreme_weight))

        # Train the model with the majority class
        train_data = normal_data[['windspeed', 'tpw', 'rainfall']].values
        history = autoencoder.fit(train_data, train_data, epochs=num_epochs, batch_size=batch_size, validation_split=0.2)

        # Use the trained autoencoder to predict the rainfall values for all data points
        test_data = df[["windspeed_tree","tpw_tree","rainfall_tree"]].values
        predicted_data = autoencoder.predict(test_data)
        data['predicted_rainfall'] = predicted_data[:, 2]

        # Calculate the error between the predicted and actual rainfall values
        data['error'] = np.abs(data['predicted_rainfall'] - data['rainfall'])

        # Classify the rainfall data into normal and extreme based on the error threshold
        # Define a range of threshold values to try
        def best_threshold():
            thresholds = np.arange(0.5, 10.0, 0.1)

            # Initialize variables to store the best threshold and the lowest error
            best_threshold = 0.0
            lowest_error = float('inf')

            # Loop over the threshold values and calculate the error for each
            for threshold in thresholds:
                # Classify the rainfall data into normal and extreme based on the threshold
                pre_df['rainfall_class'] = np.where(data['error'] > threshold, 'extreme', 'normal')

                # Calculate the accuracy of the classification
                accuracy = sum(pre_df['rainfall_class'] == pre_df['actuall_rainfall_class']) / len(data)

                # Calculate the error of the classification
                error = 1 - accuracy

                # Update the best threshold and lowest error if necessary
                if error < lowest_error:
                    best_threshold = threshold
                    lowest_error = error

            return threshold
        
        data['rainfall_class'] = np.where(data['error'] > threshold, 'Extreme', 'Normal')

        # Define the actual rainfall class based on the threshold of 11
        data['actual_rainfall_class'] = np.where(data['rainfall'] > 50, 'Extreme', 'Normal')

        result_list = []
        for index, row in data.iterrows():
            result_list.append(row['rainfall_class'] == row['actual_rainfall_class'])

        if all(result_list):
            print("found it")
            flag=True
            break
    if flag:
        break
    else:
        continue
        
print(normal_weight, extreme_weight)

C:\Users\shine\anaconda3\envs\environment\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
C:\Users\shine\anaconda3\envs\environment\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


1


TypeError: in user code:

    File "C:\Users\shine\anaconda3\envs\environment\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\shine\AppData\Local\Temp\ipykernel_1112\553882722.py", line 42, in loss  *
        normal_loss = K.mean(K.square(y_true - y_pred))

    TypeError: Input 'y' of 'Sub' Op has type float32 that does not match type int64 of argument 'x'.
